In [15]:
import numpy as np
import random
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

# Seed 고정
seed_everything(42)

# 데이터셋 로드
file_path = 'data/train.csv'
data = pd.read_csv(file_path)

# One-Hot Encoder 및 Label Encoder 생성
onehot_encoder = OneHotEncoder()
label_encoder = LabelEncoder()

# One-Hot Encoding 및 Label Encoding 적용
onehot_columns = ['대출기간', '근로기간', '주택소유상태', '대출목적']
data_onehot = pd.DataFrame(onehot_encoder.fit_transform(data[onehot_columns]).toarray())
data['대출등급_encoded'] = label_encoder.fit_transform(data['대출등급'])

# 인코딩된 데이터와 기존 데이터 합치기 및 원본 범주형 변수 제거
data_encoded = pd.concat([data, data_onehot], axis=1)
data_encoded.drop(onehot_columns + ['대출등급'], axis=1, inplace=True)

# Standard Scaling 적용
scaler = StandardScaler()
numerical_columns_for_scaling = ['대출금액', '연간소득', '부채_대비_소득_비율', '총계좌수', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액', '연체계좌수']
data_encoded[numerical_columns_for_scaling] = scaler.fit_transform(data_encoded[numerical_columns_for_scaling])

# SMOTE 적용
smote = SMOTE(random_state=42)
X = data_encoded.drop('대출등급_encoded', axis=1)
y = data_encoded['대출등급_encoded']
X.columns = X.columns.astype(str)

# 문자열 데이터가 포함된 열 식별
non_numeric_columns = X.select_dtypes(include=['object']).columns

# 문자열 데이터 열 제거
X = X.drop(non_numeric_columns, axis=1)
X_smote, y_smote = smote.fit_resample(X, y)


# 최적의 하이퍼파라미터를 사용하여 신경망 모델 인스턴스 생성
best_nn_model = MLPClassifier(
    activation='tanh',
    alpha=0.0001,
    hidden_layer_sizes=(100, 100),
    solver='sgd',
    max_iter=500,
    random_state=42
)

# 모델 훈련
best_nn_model.fit(X_smote, y_smote)

# 훈련된 모델 평가 (필요에 따라)
# 예: best_nn_model.score(X_test, y_test)

MLPClassifier(activation='tanh', hidden_layer_sizes=(100, 100), max_iter=500,
              random_state=42, solver='sgd')